In [1]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

from preprocessors import Preprocessor
from config.data_setting import *
import pandas as pd
import numpy as np
import pickle
import joblib

## Presprocessing

#### setting and save

In [2]:
processor_path = "class_preprocessor.pickle"

# symbolic_type = "onehot"
# preprocessor = Preprocessor(symbolic_type=symbolic_type, num_min=-1, num_max=1, y_min=-1, y_max=1)

# with open(processor_path, 'wb') as file:
#     pickle.dump(preprocessor, file)

#### load

In [3]:
with open(processor_path, 'rb') as file:
    preprocessor = pickle.load(file)

#### run

In [4]:
is_external = True
X_train = preprocessor.preprocess(train_path, is_external=is_external)
y_train = preprocessor.y_true
y_train = np.ravel(y_train)

## Model

### Random Forest

In [5]:
model_save_path = "save_models/rf.pkl"
result_path = "result/rf.csv"

# param_dist = {"n_estimators":sp_randint(300, 1000)}

# model = RandomForestRegressor(random_state=1008)
# model = RandomizedSearchCV(model, param_distributions=param_dist,
#                             n_iter=5, cv=10, scoring='neg_mean_absolute_percentage_error',
#                             n_jobs=4, random_state=25, return_train_score=True, verbose=True)

# model.fit(X_train, y_train)
# model = model.best_estimator_
# model

In [6]:
model = RandomForestRegressor(n_estimators=1000, random_state=1008)
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000, random_state=1008)

In [7]:
joblib.dump(model, model_save_path)
model = joblib.load(model_save_path)

### AdaBoost

In [8]:
# model_save_path = "save_models/ada.pkl"
# result_path = "result/ada.csv"

# model = AdaBoostRegressor(random_state=0)
# model.fit(X_train, y_train)
# joblib.dump(model, model_save_path)
# model = joblib.load(model_save_path)

### XGBoost

In [9]:
# model_save_path = "save_models/xgb.json"
# result_path = "result/xgb.csv"

# model = XGBRegressor()
# model.fit(X_train, y_train)
# model.save_model(model_save_path)

# model.load_model(model_save_path)

## Result

In [10]:
X_test = preprocessor.preprocess(public_path, is_external=is_external)
y_test = model.predict(X_test)
y_test = preprocessor.denormalizeY(y_test)

# save result
df = pd.read_csv(sub_template_path)
df = pd.DataFrame(df)
df[price_col] = y_test
df.to_csv(result_path, index=False)